# Obliczanie wartości sinusa w dziedzinie liczb zespolonych

### Podstawowe funkcje i parametry

In [97]:
import Base.+, Base.*, Base.-, Base./

using Plots

plotly()

Prec = BigFloat
Comp = Tuple{Prec, Prec}

SIN_MAX_ITER = 10
COS_MAX_ITER = 10
SINH_MAX_ITER = 50
COSH_MAX_ITER = 50

PRECISION = 128

setprecision(PRECISION)

# Podstawowe operacje na liczbach zespolonych
function +(x::Comp, y::Comp)
    return x[1] + y[1], x[2] + y[2]
end

function *(x::Comp, y::Comp)
    return  x[1] * y[1] - x[2] * y[2], x[1] * y[2] + x[2] * y[1]
end

function -(x::Comp)
    return -x[1], -x[2]
end

function /(x::Comp, n::Int64)
    return x[1] / n, x[2] / n
end

# Kwadrat liczby zespolonej
function square(x::Comp)
    return x * x
end

# Funkcja obliczająca n-tą potęgę liczby zespolonej,
# korzystając z algorytmu szybkiego potęgowania.
# Ogranicza do minimum wykonywaną liczbę mnożeń (w tym błąd).
function power_complex(x::Comp, n::Int64)
    if n > 0
        if n & 1 == 1
            return square(power_complex(x, n>>=1)) * x
        else
            return square(power_complex(x, n>>=1))
        end
    else
        return Comp((1, 0))
    end
end

# Funkcja obliczająca wartość symbolu newtona
function newton(n::Int64, k::Int64)
    if n < k return 0 end
    if k < 0 return 0 end
    Σ = Prec(1)
    for i = 1:(n-k)
         Σ = Σ * (i + k) / i
    end
    return Σ
end

# Funkcja testująca
function test(f::Function, z::Comp, max_iter::Int64)
    setprecision(1024)
    result = sin(z[1] + z[2] * im)
    res = BigFloat(real(result), 128), BigFloat(imag(result), 128)
    setprecision(128)
    
    f(z, res, max_iter)
end

# Funkcja uśredniająca
function nor(a::Array)
    b = a
    s = Prec(1)
    for i = 1:size(a)[1]
        s *= a[i]
        b[i] = s
    end
    return b
end

nor (generic function with 1 method)

## Sposób 1
Podstawowym pomysłem jest obliczanie sinusa z szeregu Taylora, kolejno obliczając potęgi i sumując liczby zespolone. Podejście jest bardzo naiwne, ale dla liczb o małym module okazuje się dość dobre.

In [139]:
MAX_ITER = 30

function sin_complex_1(z::Comp, res, max_iter)
    Σ  = z
    _z = z
    Δ_real = Array{BigFloat}(max_iter)
    Δ_imag = Array{BigFloat}(max_iter)
    for i = 1:max_iter
        _z = - _z * z / (2 * i) * z / (2 * i + 1)
        @printf "%3d: Błąd względny: %.5e %.5e \t Różnica: %.5e %.5e\n" i abs((Σ[1] - res[1])/res[1]) abs((Σ[2] - res[2])/res[1]) abs((Σ[1] + _z[1]) - Σ[1]) abs((Σ[2] + _z[2]) - Σ[2])
        Σ += _z
        Δ_real[i] = abs(Σ[1] - res[1])
        Δ_imag[i] = abs(Σ[2] - res[2])
    end
    return Σ, Δ_real, Δ_imag
end

x = BigFloat("439231.51802749848917298471928741")
s = BigFloat(0)
n = 1000
for i = 1:n
    s += (nextfloat(x) - x) / x
    x = nextfloat(x)
end
u = s/n

u

3.507817386213457031626733235400812795353e-39

### Przykład 1.1
Test naiwnej funkcji sinus na liczbie zespolonej $2+3i$. 

In [141]:
res = test(sin_complex_1, Comp((2, 3)), 30)

plot(abs.(res[2][2:end] ./ res[2][1:end-1]), label="Real", ylabel="Iloraz błędów", xlabel="Iteracje")
plot!(abs.(res[3][2:end] ./ res[3][1:end-1]), label="Imaginary")

  1: Błąd względny: 7.81528e-01 7.83102e-01 	 Różnica: 7.66667e+00 1.50000e+00
  2: Błąd względny: 5.59471e-02 6.19248e-01 	 Różnica: 1.01667e+00 4.97500e+00
  3: Błąd względny: 1.67004e-01 7.57996e-02 	 Różnica: 1.30040e+00 8.82738e-01
  4: Błąd względny: 2.49536e-02 2.06271e-02 	 Różnica: 2.37428e-01 1.55432e-01
  5: Błąd względny: 9.82139e-04 3.64843e-03 	 Różnica: 6.16397e-03 3.29663e-02
  6: Błąd względny: 3.08812e-04 4.73257e-05 	 Różnica: 2.73344e-03 5.82462e-04
  7: Błąd względny: 1.02226e-05 1.63001e-05 	 Różnica: 9.83653e-05 1.42328e-04
  8: Błąd względny: 5.22402e-07 7.52761e-07 	 Różnica: 4.47100e-06 6.95597e-06
  9: Błąd względny: 3.40087e-08 7.08071e-09 	 Różnica: 3.09435e-07 5.51816e-08
 10: Błąd względny: 2.07345e-10 1.05290e-09 	 Różnica: 2.10713e-09 9.49792e-09
 11: Błąd względny: 2.28289e-11 1.53843e-11 	 Różnica: 2.04426e-10 1.43824e-10
 12: Błąd względny: 4.98299e-13 3.26527e-13 	 Różnica: 4.58003e-12 2.88998e-12
 13: Błąd względny: 2.00529e-15 1.08381e-14 	 Różnic

Widać, że otrzymane wartości są bardzo bliskie dokładności maszynowej, a zbieżność funkcji jest mniej więcej liniowa (widać to z wykresu). Jednak problemy pojawiają się, gdy próbujemy obliczać sinusa z liczb o dużym module.

### Przykład 1.2

Na poniższym przykładzie widać, że dla liczb zespolonych o nieco większym module (w tym przypadku: 10 razy większym) algorytm traci na zbieżności (potrzeba ponad 2 razy więcej iteracji, by zatrzymać się w pukcie stałym) oraz dokładności. W tym przypadku błąd względny jest ok. 100-krotnie większy.

In [96]:
test(sin_complex_1, Comp((20, 30)), 75)[1]

# plot(abs.(res[2][2:end] ./ res[2][1:end-1]), label="Real", ylabel="Iloraz błędów", xlabel="Iteracje")
# plot!(abs.(res[3][2:end] ./ res[3][1:end-1]), label="Imaginary")

7.8944818705e-39 3.0950504709e-39
1: 9.9999999842e-01 4.4699510925e-01 
2: 9.9999997758e-01 4.4699521124e-01 
3: 1.0000026434e+00 4.4699702084e-01 
4: 1.0000513158e+00 4.4696515759e-01 
5: 9.9992495533e-01 4.4628935228e-01 
6: 9.9432145123e-01 4.4509531358e-01 
7: 9.7415669653e-01 4.7427238769e-01 
8: 1.0658115185e+00 6.1686886822e-01 
9: 1.7001483244e+00 5.0374738835e-01 
10: 2.1321069604e+00 1.4433119142e+00 
11: 2.0585883854e+00 4.3916913893e+00 
12: 1.1447593457e+01 1.5327164064e+00 
13: 8.0077261939e+00 2.1801961398e+01 
14: 2.4064967922e+01 2.9199475905e+01 
15: 5.0853531864e+01 8.2074871970e+00 
16: 2.1029598387e+01 5.6360667388e+01 
17: 4.0059294863e+01 4.6518541273e+01 
18: 5.4123780815e+01 1.2210989476e+01 
19: 1.1314608127e+01 4.3413518839e+01 
20: 2.4568087958e+01 2.1602700215e+01 
21: 2.0010150911e+01 8.2780553856e+00 
22: 7.4960785896e-01 1.3061314650e+01 
23: 6.3596348070e+00 3.4771231468e+00 
24: 2.9810597767e+00 2.1874971555e+00 
25: 3.4710958712e-01 1.7082893574e+00 


(4.878083108112134954602104644251684351603e+12, 2.180479290372453909365240221267619439517e+12)

### Przykład 1.3
Test naiwnego sinusa dla argumentu $200+300i$. Zbieżność w tym przypadku jest bardzo słaba (~350 iteracji), a błąd względny jest rzędu $\sim10^{25} \cdot u$, gdzie $u$ to precyzja arytmetyki.

In [107]:
test(sin_complex_1, Comp((200, 300)), 350)[1]

1: 1.0000000000e+00 5.5787150213e-01 
2: 1.0000000000e+00 5.5787150213e-01 
3: 1.0000000000e+00 5.5787150213e-01 
4: 1.0000000000e+00 5.5787150213e-01 
5: 1.0000000000e+00 5.5787150213e-01 
6: 1.0000000000e+00 5.5787150213e-01 
7: 1.0000000000e+00 5.5787150213e-01 
8: 1.0000000000e+00 5.5787150213e-01 
9: 1.0000000000e+00 5.5787150213e-01 
10: 1.0000000000e+00 5.5787150213e-01 
11: 1.0000000000e+00 5.5787150213e-01 
12: 1.0000000000e+00 5.5787150213e-01 
13: 1.0000000000e+00 5.5787150213e-01 
14: 1.0000000000e+00 5.5787150213e-01 
15: 1.0000000000e+00 5.5787150213e-01 
16: 1.0000000000e+00 5.5787150213e-01 
17: 1.0000000000e+00 5.5787150213e-01 
18: 1.0000000000e+00 5.5787150213e-01 
19: 1.0000000000e+00 5.5787150213e-01 
20: 1.0000000000e+00 5.5787150213e-01 
21: 1.0000000000e+00 5.5787150213e-01 
22: 1.0000000000e+00 5.5787150213e-01 
23: 1.0000000000e+00 5.5787150213e-01 
24: 1.0000000000e+00 5.5787150213e-01 
25: 1.0000000000e+00 5.5787150213e-01 
26: 1.0000000000e+00 5.5787150213e

330: 7.2137272088e-14 5.3352086408e-14 
331: 7.2137290341e-14 5.3352147971e-14 
332: 7.2137309138e-14 5.3352149981e-14 
333: 7.2137311797e-14 5.3352145130e-14 
334: 7.2137310792e-14 5.3352143873e-14 
335: 7.2137310345e-14 5.3352144002e-14 
336: 7.2137310329e-14 5.3352144134e-14 
337: 7.2137310363e-14 5.3352144153e-14 
338: 7.2137310371e-14 5.3352144146e-14 
339: 7.2137310370e-14 5.3352144143e-14 
340: 7.2137310370e-14 5.3352144143e-14 
341: 7.2137310369e-14 5.3352144144e-14 
342: 7.2137310370e-14 5.3352144144e-14 
343: 7.2137310370e-14 5.3352144144e-14 
344: 7.2137310370e-14 5.3352144144e-14 
345: 7.2137310370e-14 5.3352144144e-14 
346: 7.2137310370e-14 5.3352144144e-14 
347: 7.2137310370e-14 5.3352144144e-14 
348: 7.2137310370e-14 5.3352144144e-14 
349: 7.2137310370e-14 5.3352144144e-14 
350: 7.2137310370e-14 5.3352144144e-14 


(-8.481578605005944143236848271012263331388e+129, 4.731630996849587555505121909471980493951e+129)

## Podejście 2
Poniżej przedstawiam implementację algorytmu obliczania sinusa zespolonego korzystając ze wzoru (11) z raportu.

In [123]:
function sin_complex_2(z, res, max_iter)
    x = z[1]
    y = z[2]
    as = a = Prec(1)
    bs = b = z[1]
    cs = c = Prec(1)
    ds = d = z[2]
    for i = 1:max_iter
        a = - a * z[1] / (2  * i - 1) * z[1] / (2 * i)
        b = - b * z[1] / (2 * i) * z[1] / (2 * i + 1)
        c = c * z[2] / (2  * i - 1) * z[2] / (2 * i)
        d = d * z[2] / (2 * i) * z[2] / (2 * i + 1)
        as += a
        bs += b
        cs += c
        ds += d
        @printf "%3d: Błąd względny: %.5e %.5e \t Różnica: %.5e %.5e\n" i abs((cs * bs - res[1])/res[1]) abs((as * ds - res[2])/res[1]) abs(x - cs * bs) abs(y - as * ds)
        x = cs * bs
        y = as * ds
    end
    return cs * bs, as * ds
end

sin_complex_2 (generic function with 2 methods)

### Przykład 2.1
Zastosowanie powyższego algorytmu dla argumentu $z = 2 + 3i$

In [124]:
test(sin_complex_2, Comp((20, 30)), 75)[1]

  1: Błąd względny: 1.00000e+00 4.46995e-01 	 Różnica: 5.92333e+05 9.01500e+05
  2: Błąd względny: 9.99822e-01 4.46721e-01 	 Różnica: 8.67702e+08 1.33990e+09
  3: Błąd względny: 1.04905e+00 5.23811e-01 	 Różnica: 2.40159e+11 3.76052e+11
  4: Błąd względny: 3.19768e+00 6.21136e+00 	 Różnica: 2.07159e+13 3.28547e+13
  5: Błąd względny: 1.46727e+02 2.34255e+02 	 Różnica: 7.31347e+14 1.17302e+15
  6: Błąd względny: 2.43292e+03 3.94381e+03 	 Różnica: 1.25837e+16 2.03810e+16
  7: Błąd względny: 2.20190e+04 3.60011e+04 	 Różnica: 1.19278e+17 1.94855e+17
  8: Błąd względny: 1.17736e+05 1.94132e+05 	 Różnica: 6.81737e+17 1.12261e+18
  9: Błąd względny: 3.98066e+05 6.61958e+05 	 Różnica: 2.51613e+18 4.17608e+18
 10: Błąd względny: 8.97925e+05 1.50699e+06 	 Różnica: 6.32195e+18 1.05803e+19
 11: Błąd względny: 1.41261e+06 2.39578e+06 	 Różnica: 1.12710e+19 1.90380e+19
 12: Błąd względny: 1.60941e+06 2.76359e+06 	 Różnica: 1.47417e+19 2.51678e+19
 13: Błąd względny: 1.37251e+06 2.39226e+06 	 Różnic

4.87808310811213495460210464425189299018e+12

In [7]:
function square(x)
    return x * x
end

# Funkcja obliczająca szybko kolejne potęgi. Zachowuje precyzję x (typ x = typ wartości zwracanej)
function power(x, n)
    if n > 0
        if n & 1 == 1
            return square(power(x, n>>=1)) * x
        else 
            return square(power(x, n>>=1))
        end
    else 
        return 1
    end
end

# Funkcja obliczająca sinusa z liczby rzeczywistej x korzystając
# z rozwinięcia w szereg Taylora długości n
function sin_real(x::Prec)
    δx = x
    Σ = x
    for i = 1:SIN_MAX_ITER
        δx = - δx * x / (2 * i) * x / (2 * i + 1)
        Σ += δx
    end
    return Σ
end

# Funkcja obliczająca cosinusa z liczby rzeczywistej x korzystając
# z rozwinięcia w szereg Taylora długości n
function cos_real(x::Prec)
    δx = Prec(1)
    Σ = Prec(1)
    for i = 1:COS_MAX_ITER
        δx = Prec(- δx  * x / (2i - 1) * x / (2i))
        Σ += δx
    end
    return Σ
end

function sin_nx(nx, n)
    Σ = Prec(0)
    x = Prec(nx) / prec(n)
    cos = cos_real(x)
    sin = sin_real(x)
    for i = 0:div(n, 2)
        Σ += (i % 2 == 0 ? 1 : -1) * newton(n, 2 * i + 1) * power(cos, n - 2 * i - 1) * power(sin, 2 * i + 1)
    end
    return Σ
end

function cos_nx(nx, n)
    Σ = Prec(0)
    x = Prec(nx) / Prec(n)
    cos = cos_real(x)
    sin = sin_real(x)
    for i = 0:div(n, 2)
        Σ += (i % 2 == 0 ? 1 : -1) * newton(n, 2 * i) * power(cos, n - 2 * i) * power(sin, 2 * i)
    end
    return Σ
end


cos_nx (generic function with 1 method)

# Sposób 3

### Obliczanie sinusa metodą rekurencyjnych wzorów bez użycia pochodnych

In [8]:
setprecision(BigFloat, 128)

function ilosc_cyfr(x)
    ilosc = 0
    while x > 0
        ilosc = ilosc + 1
        x = x / 10
    end
    
    return ilosc
end

function msqrt(x, iter)
    start = x / 2 # (x / (10^ilosc_cyfr(x))) + 1 # Mozna ulepszyc wykorzystujac 10^ilosc_cyfr(x)
    
    x_n = BigFloat(start)
    for i = 0:iter
       x_n = 1/2 * (x_n + x/x_n)
    end
    
    return x_n
end

function sincos(iter)
    cn = -1 # cos pi
    sn = 0 # sin pi
    
    for i = 0:iter
        cn = msqrt((1 + cn)/2, 1000)
        sn = msqrt((1 - sn)/2, 1000)
    end
    
    @show cn
    @show sn
end

function rozwiniecie_binarne(x)
    ulam = BigFloat(x)
    t = Int32[]
    
    for i = 1:1000
        if ulam == 0
            @printf "1"
            break
        end
            
        ulam = BigFloat(BigFloat(ulam) * BigFloat(2))
        if (ulam >= 1)
            ulam = BigFloat(ulam - 1)
            push!(t, 1)
            @printf "%lf \t 1 \t %d\n" ulam i
        else
            push!(t, 0)
            @printf "%lf \t 0 \t %d\n" ulam i
        end
    end
    
    return t
end

# t jest rozwinieciem binarnym α/π
function ograniczenie(t, maxiter)
    Σ = BigFloat(0)
    l = length(t)
    
    cn = -1
    sn = 0
    for i = 1:l
        cn = msqrt((1 + cn)/2, 100)
        sn = msqrt((1 - cn)/2, 100)
        # @show cn sn
        
        @show Σ
        Σ = Σ + t[i] * BigFloat(π)/(2^i)
    end
    
    return BigFloat(sin(Σ)), BigFloat(sin(Σ + π/(2^l)))
end

# sinus bez pochodnych z wzorami rekurencyjnymi
function sinwd(α)
    x = BigFloat(BigFloat(α)/BigFloat(π))
    t = rozwiniecie_binarne(x)
    return ograniczenie(t, 500)
end

# Szybka aproksymacja sinusa, która ma na celu wydajność
# Kąt alpha musi być znormalizowany w postaci od [-π, π]
function rpdsin(angle)
    # normalizujemy kąt do przedziału [-π, π]
    if (angle < -π)
        angle += 2*π
    elseif (angle > π)
        angle -= 2*π
    end
        
    if (angle < 0)
        sin = 1.27323954 * angle + .405284735 * angle * angle;

        if (sin < 0)
            sin = .225 * (sin *-sin - sin) + sin;
        else
            sin = .225 * (sin * sin - sin) + sin;
        end
    else
        sin = 1.27323954 * angle - 0.405284735 * angle * angle;

        if (sin < 0)
            sin = .225 * (sin *-sin - sin) + sin;
        else
            sin = .225 * (sin * sin - sin) + sin;
        end
    end
end
    
@show msqrt(5, 1000)
@show sincos(100)

@show "Porównanie szybkiego liczenia z wzorami rekurencyjnymi"
@show sinwd(0.49)
@show rpdsin(1)

msqrt(5, 1000) = 2.23606797749978969640917366873127623544
cn = BigFloat(NaN, 128)
sn = 5.000000000000000000000000000001788544277e-01
sincos(100) = 5.000000000000000000000000000001788544277e-01
"Porównanie szybkiego liczenia z wzorami rekurencyjnymi" = "Porównanie szybkiego liczenia z wzorami rekurencyjnymi"
0.311944 	 0 	 1
0.623887 	 0 	 2
0.247775 	 1 	 3
0.495550 	 0 	 4
0.991099 	 0 	 5
0.982198 	 1 	 6
0.964396 	 1 	 7
0.928792 	 1 	 8
0.857584 	 1 	 9
0.715168 	 1 	 10
0.430337 	 1 	 11
0.860674 	 0 	 12
0.721348 	 1 	 13
0.442696 	 1 	 14
0.885392 	 0 	 15
0.770783 	 1 	 16
0.541567 	 1 	 17
0.083134 	 1 	 18
0.166268 	 0 	 19
0.332535 	 0 	 20
0.665071 	 0 	 21
0.330142 	 1 	 22
0.660283 	 0 	 23
0.320566 	 1 	 24
0.641132 	 0 	 25
0.282264 	 1 	 26
0.564528 	 0 	 27
0.129056 	 1 	 28
0.258113 	 0 	 29
0.516226 	 0 	 30
0.032451 	 1 	 31
0.064903 	 0 	 32
0.129806 	 0 	 33
0.259612 	 0 	 34
0.519224 	 0 	 35
0.038447 	 1 	 36
0.076894 	 0 	 37
0.153789 	 0 	 38
0.307577 	 0 	 3

0.8421677211675824

<h2>Test #1</h2>
Sprawdzamy, dla jakiego n skracanie przedziału \[0, &pi;/2) korzystając ze wzoru na sinus zwielokrotnionego kąta ma sens.

In [9]:
tests = [Float32(0.321563716), Float32(0.034321248), Float32(1.0523452), Float32(0.00000123456), Float32(1.000003123)]
range = 10000
max_n = 30
Prec = BigFloat

using PyPlot
xPlot = BigFloat[]
yPlot = BigFloat[]

for k = 1:max_n
    Σ² = 0
    for i = 1:5
    curr_test = tests[i]
        for j = 1:range
            curr_test = nextfloat(curr_test)
            target = sin(Float64(curr_test))
            Δ = target - sin_nx(curr_test, k, 10)
            Σ² += Δ * Δ
        end
    end
    @printf "n = %d, σ = %e \n" k sqrt(Σ² / range)
    push!(xPlot, k)
    push!(yPlot, sqrt(Σ² / range))
end

plot(xPlot, yPlot)

LoadError: [91mArgumentError: Module PyPlot not found in current path.
Run `Pkg.add("PyPlot")` to install the PyPlot package.[39m

In [10]:
k = 100
n = k / 2
x = Float64(1.0)
while x > 0.5
    x *= k * k / (2n - 1) / (2n)
    n += 1
end

@show n

n = 57.0


57.0

In [11]:
exact_sinh = Prec("10.0178749274099018989745936194658280601781041231828634644056532510463926051808870905252214580081921788136031436005276604654731845463086661965545666809253301856979834214884134800415226711700580868934289537240194339117071445876738421988703293310666662074873025393922419074202969904427384978934547999669783367996478777737387515158099245608102468799707910743158654767792319545286680090845441765466693103005564580755152769830821020898523711510472417274773015585649780529003697102125038131242050195636504635373129836335784769819562604683637035572605572218826029004693336598037776759038456992068734019962315072590278266626183660525078196734326386033216758620007131926640350061262077283570845419590562070693093134140875714806032858928269295308563453677379062041720184214492393331096756140382287179704817388080282537490855424255503455845715185625683972134857957278057234162657813792323741991")

function sinh_real(x::Prec)
    Σ = x
    δx = x
    for i = 1:SINH_MAX_ITER
        δx = δx * x / (2 * i) * x / (2 * i + 1)
        Σ += δx
        @show Σ - exact_sinh
    end
    return Σ
end

function cosh_real(x::Prec)
    Σ = Prec(1)
    δx = Prec(1)
    for i = 1:COSH_MAX_ITER
        δx = δx * x / (2 * i - 1) * x / (2 * i)
        Σ += δx
        @show Σ
    end
    return Σ
end

function sin_complex_n(z, n)
    Σ_r = Prec(0)
    Σ_i = Prec(0)
    cos = cos_complex(z)
    sin = sin_complex(z)
    for i = 0:div(n, 2)
        σ = multiply_complex(power_complex(cos, n - 2 * i - 1), power_complex(sin, 2 * i + 1))
        Σ_r += (i % 2 == 0 ? 1 : -1) * newton(n, 2 * i + 1) * σ[1]
        Σ_i += (i % 2 == 0 ? 1 : -1) * newton(n, 2 * i + 1) * σ[2]
    end
    return Σ_r, Σ_i
end


function sin_complex(z)
    return sin_real(z[1]) * cosh_real(z[2]), cos_real(z[1]) * sinh_real(z[2])
end

z = Prec(1), Prec(3)
exact_result = Prec("8.471645454300149424897583639976881207279850599273339864612249252167060156811179"), 
            Prec("5.41268092317819278427941063379302764217781251017215863262723484500606267062513")
z = sin_complex(z)
@show z[1] - exact_result[1]
@show z[2] - exact_result[2]

@show exact_sinh - nextfloat(exact_sinh)

Σ = 5.500000000000000000000000000000000000000
Σ = 8.875000000000000000000000000000000000000
Σ = 9.887499999999999999999999999999999999991
Σ = 1.005022321428571428571428571428571428569e+01
Σ = 1.006649553571428571428571428571428571427e+01
Σ = 1.006760501217532467532467532467532467532e+01
Σ = 1.006765987639592550306836021121735407448e+01
Σ = 1.006766193380419803410874839446268017695e+01
Σ = 1.006766199431620604972758334102871917999e+01
Σ = 1.006766199574938518693960837923686220901e+01
Σ = 1.006766199577730426104114133452663122904e+01
Σ = 1.006766199577775946333627502401505137609e+01
Σ = 1.006766199577776576613728456740796796275e+01
Σ = 1.006766199577776584117062991911502649355e+01
Σ = 1.006766199577776584194683693999475468525e+01
Σ = 1.006766199577776584195387914078902641279e+01
Σ = 1.006766199577776584195393562903069169402e+01
Σ = 1.00676619957777658419539360325181321603e+01
Σ = 1.006766199577776584195393603510091093993e+01
Σ = 1.006766199577776584195393603511581158673e+01
Σ = 1.0067661

-4.701977403289150031874946148888982711275e-38